In [0]:
!pip install ggplot
!pip install matplotlib
!pip install bokeh
!pip install dask
!pip install statsmodels
!pip install cloudpickle
!pip install requests
!pip install tpot
!pip install lightgbm
!pip install networkx

In [0]:
import seaborn as sns
from ggplot import *
from matplotlib import pyplot as plt
import bokeh
import tqdm

import pandas as pd
import dask.dataframe as dd
import numpy as np
import scipy as sc
import statsmodels as sm
import networkx as nx

import sklearn as sk
import tensorflow as tf
import keras
from xgboost import XGBClassifier as xgb
from lightgbm import LGBMClassifier as lgbm
from sklearn.ensemble import RandomForestClassifier as rf
import tpot

import sys
import os
import gc
import re

# target variables: Sample Type, New Tumor Event, T-stage, N-stage
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA as pca
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.feature_selection import SelectFdr as fdr
from sklearn.manifold import TSNE


pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.1f}'.format

In [0]:
df_copy_number = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_CNV.txt", sep="\t")
df_phenotypes = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_Phenotype_Metadata.txt", sep="\t")

In [0]:
df_copy_number_plus = df_copy_number.loc[df_copy_number.Strand=='+']
df_copy_number_plus.drop('Strand', axis=1, inplace=True)
df_copy_number_min = df_copy_number.loc[df_copy_number.Strand=='-']
df_copy_number_min.drop('Strand', axis=1, inplace=True)
df_copy_number.drop('Strand', axis=1, inplace=True)

# Feature manipulation


In [0]:
df_copy_number=df_copy_number_plus

In [0]:
#df_copy_number['Strand'] = df_copy_number['Strand'].apply(lambda x: -1 if x=='-' else 1)
df_copy_number['GeneDiff'] = df_copy_number['Stop']-df_copy_number['Start']
df_copy_number.Chr = df_copy_number.loc[(~df_copy_number.Chr.isna()) & (df_copy_number.Chr.str.contains('chr'))].Chr\
                                    .apply(lambda x: re.sub(r'chr', '', x))

In [0]:
df_copy_number['Start'] = df_copy_number.Start.astype('str')
df_copy_number['Stop'] = df_copy_number.Start.astype('str')
df_copy_number['Chr'] = df_copy_number.Start.astype('str')
df_copy_number_concat = df_copy_number.copy()
df_copy_number_concat['GenX'] = df_copy_number_concat[['Gene', 'Chr', 'Start', 'Stop']].apply(lambda x: '|'.join(x), axis=1)
df_copy_number_concat = df_copy_number_concat.drop(['Gene', 'Chr', 'Start', 'Stop'], axis=1)
df_copy_number_concat = df_copy_number_concat.drop(['GeneDiff'], axis=1)
gc.collect()

In [0]:
df_copy_number.drop(['Chr', 'Start', 'Stop'], axis=1, inplace=True)

In [0]:
df_copy_number.GeneDiff = df_copy_number.GeneDiff/df_copy_number.GeneDiff.max()
df_copy_number.GeneDiff = pd.np.log10(df_copy_number.GeneDiff+1)

In [0]:
df_copy_number.GeneDiff.plot.kde(figsize=(12,8))

In [0]:
replacementValue = df_copy_number.GeneDiff.median()
df_copy_number.GeneDiff.fillna(replacementValue, inplace=True)

In [0]:

df_copy_number_weighted_by_GeneDiff = df_copy_number.iloc[:,1:].div(df_copy_number.GeneDiff, axis=0)
df_copy_number_weighted_by_GeneDiff.drop(['GeneDiff'], axis=1, inplace=True)
df_copy_number.drop(['GeneDiff'], axis=1, inplace=True)
cols=df_copy_number.Gene
df_copy_number.set_index('Gene')
df_copy_number.drop('Gene', axis=1, inplace=True)

In [0]:
df_copy_number_transposed = df_copy_number[df_copy_number.columns[~df_copy_number.\
                                          columns.\
                                              isin(['Chr', 'Start', 'Stop', 'Strand', 'GeneDiff'])]].T
df_copy_number_transposed.columns = cols

del df_copy_number
gc.collect()

In [0]:
df_copy_number_transposed = df_copy_number_weighted_by_GeneDiff[df_copy_number_weighted_by_GeneDiff.columns[~df_copy_number_weighted_by_GeneDiff.\
                                          columns.\
                                              isin(['Chr', 'Start', 'Stop', 'Strand', 'GeneDiff'])]].T

df_copy_number_transposed.columns = cols

del df_copy_number
gc.collect()


In [0]:

df_copy_number_transposed.index.rename('Sample', inplace=True)
df_copy_number_transposed.reset_index(inplace=True)
df_copy_number_transposed.index=df_copy_number_transposed.Sample
df_copy_number_transposed.drop('Sample', axis=1, inplace=True)

In [0]:
df_phenotypes.columns

In [0]:
target_variable = 'Response To Therapy'
target_map = {
 "Complete Response":0,
 "Clinical Progressive Disease":1,        
 "Radiographic Progressive Disease":1,    
 "Stable Disease":1,                      
 "Partial Response":0                    
}
df_phenotypes[target_variable].describe()
print(df_phenotypes[[target_variable, 'SampleID']].groupby(by=target_variable).count())

In [0]:
le = LabelEncoder()
merged = df_copy_number_transposed.merge(df_phenotypes.loc[df_phenotypes['Drug Therapy Type']=='Immunotherapy'][['SampleID', target_variable]], how='inner', left_index=True, right_on='SampleID')
merged = merged.loc[~merged[target_variable].isna()]

merged[target_variable] = merged[target_variable].apply(lambda x: target_map[x])

y= le.fit_transform(merged[target_variable].astype('str'))
x= merged.loc[:, merged.columns != target_variable]
x.drop('SampleID', axis=1, inplace=True)
x.drop(target_variable, axis=1, inplace=True)
#del merged 
gc.collect()

# Feature normalisation

In [0]:
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
# RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True)[source]
x_normalised = scaler.fit_transform(x)

# Clustering



## Parallel coordinates

## Get patient-to-patient similarity matrix

The basic ingredient to find patient clusters 


## Get gene-to-gene similarity matrix

The basic ingredient to find gene clusters

## Space embedding using t-SNE/LLE/MDS

Embedding in 3-dimensional space allows for visualisation. To identify clusters per target variable we need to attach the clinical data. This is useful because a priori we do not know what target variables leads to the best seperation.

To avoid computational complexity issues: 
* base t-SNE on exemplars
* apply PCA/LDA or some other dimension reducer before apply t-SNE
* use hierarchical t-SNE: https://github.com/DmitryUlyanov/Multicore-TSNE, https://github.com/danielfrg/tsne

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca_result = pca.fit_transform(x)


df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]

print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

## HDBSCAN

# Dimension reduction

## FDR/Wilcoxon/Mann-Whitney U



In [0]:
sel_fdr = fdr(alpha=0.5)
x_final = sel_fdr.fit_transform(x_normalised, y)
print(x_final.shape)

## PCA

In [0]:
#x = x_normalised
PCA = pca(n_components=250)
PCA.fit(x)
variance = np.cumsum(np.round(PCA.explained_variance_ratio_, decimals=3)*100)
plt.ylabel('Variance')
plt.xlabel('Features')
plt.plot(variance, label='PCA')
plt.title('PCA analysis')

In [0]:
x_final = pca(n_components=50).fit_transform(x, y)

# Classification

In [0]:
try:
  x.dropna(how='all', axis=1, inplace=True)
  x = x * 1.0
except Exception as e:
  print(e)
for col in tqdm.tqdm(x.columns):
   if 'object' in str(x[col].dtypes):
      try:
        x[col] = x[col].astype('float')
      except:
        try:
          x[col] = x[col].astype('int')
        except:
          print(col)
          x[col] = x[col].astype('category')
      

In [0]:
#x_final = x
model = lgbm(boosting_type='goss')
print("mean acc:{}, var acc:{}".format(np.mean(cross_val_score(model, x_final, y, cv=10)),
                                       np.var(cross_val_score(model, x_final, y, cv=10))))
model.fit(x_final,y)
feature_importances = model.feature_importances_

## Observations

Basic,  the CNV data only contains data from patients with metastasis.

### Response 

First we reduced the number of response types to two types,  **any** response, and **no** response, for **all** drug therapies, this gives 
about $84$ samples ($0:37, 1:47$)

* FDR with ANOVA did not work well for any value of $\alpha$
* after dimension reduction using PCA with about 100 remaining dimensions, the response can be predicted with about 70%
* using chromosome length as weight for the copy number has no effect on the accuracy of the prediction

 
 Now we limit the predictor to immunotherapy only, this gives $38$ samples ($0:15, 1: 19$).
 
* FDR with ANOVA did not work well for any value of $\alpha$
* after dimension reduction using PCA with about 30 remaining dimensions, the response can be predicted with about 40% --> with no useful features for LGBM
* using chromosome length as weight for the copy number has no effect on the accuracy of the prediction


 Now we limit the predictor to non immunotherapy only, this gives $50$ samples ($0:22, 1: 28$).
 
* FDR with ANOVA did not work well for any value of $\alpha$
* after dimension reduction using PCA with about 50 remaining dimensions, the response can be predicted with about 40%
* using chromosome length as weight for the copy number has no effect on the accuracy of the prediction


 Now we limit the predictor to non immunotherapy only, using only the **+** strand
 
* FDR with ANOVA did not work well for any value of $\alpha$
* after dimension reduction using PCA with about 50 remaining dimensions, the response can be predicted with about 69%
* using chromosome length as weight for the copy number has no effect on the accuracy of the prediction

Now we limit the predictor to non immunotherapy only, using only the **-** strand
 
* FDR with ANOVA did not work well for any value of $\alpha$
* after dimension reduction using PCA with about 50 remaining dimensions, the response can be predicted with about 63
* using chromosome length as weight for the copy number has no effect on the accuracy of the prediction

 


# Data output 

* important genes 
* reduce, normalised, tranposed dataset

Format: just  `.csv` 

# New Section